In [2]:
import numpy as np
import pandas as pd
import os
from tvb.simulator.lab import models, connectivity, coupling, integrators, monitors, noise
from tvb.simulator.simulator import Simulator

/opt/anaconda3/envs/tvb_env/lib/python3.10/site-packages/tvb/datatypes/surfaces.py:60: UserWarning: Geodesic distance module is unavailable; some functionality for surfaces will be unavailable.
  warnings.warn(msg)


In [4]:
np.random.seed(50)

In [6]:
def focal_dlpfc_lesion(conn, node_idx=18, weight_scale=0.3, tract_scale=1.5):
    W = conn.weights.copy()
    D = conn.tract_lengths.copy()
    W[node_idx, :] *= weight_scale
    W[:, node_idx] *= weight_scale
    D[node_idx, :] *= tract_scale
    D[:, node_idx] *= tract_scale
    conn_lesion = connectivity.Connectivity(weights=W, tract_lengths=D,
                           region_labels=conn.region_labels,
                           centres=conn.centres, speed=conn.speed.copy())
    conn_lesion.configure()
    return conn_lesion

In [8]:
def damp_node_dynamics(wc_model, node_idx=18, num_regions=76,
                       P_reduction=0.5, c_ie_increase=1.2):
    for pname in ('P', 'c_ie'):
        val = getattr(wc_model, pname)
        if val.size == 1:
            setattr(wc_model, pname, np.repeat(val[0], num_regions))
    wc_model.P[node_idx]    *= P_reduction
    wc_model.c_ie[node_idx] *= c_ie_increase

In [20]:
n_subjects = 50
results = []
output_dir = "tbi_stronggroup_results"
os.makedirs(output_dir, exist_ok=True)

In [22]:
for i in range(n_subjects):
    print(f"Running TBI subject {i+1}...")

    conn = connectivity.Connectivity.from_file("connectivity_76.zip")
    conn.configure()
    conn_lesion = focal_dlpfc_lesion(conn)

    wc = models.WilsonCowan(
        c_ee=np.array([np.random.normal(20.0, 1.0)]),
        c_ei=np.array([np.random.normal(12.0, 0.5)]),
        c_ie=np.array([np.random.normal(18.0, 1.0)]),
        c_ii=np.array([np.random.normal(3.0, 0.3)]),
        P=np.array([np.random.normal(0.45, 0.02)]),
        tau_e=np.array([10.0]),
        tau_i=np.array([25.0]),
    )
    damp_node_dynamics(wc, node_idx=18, num_regions=conn_lesion.number_of_regions)


    sim = Simulator(
        model=wc,
        connectivity=conn_lesion,  
        coupling=coupling.Scaling(a=np.array([0.003])),
        integrator=integrators.HeunDeterministic(dt=0.1),
        monitors=(monitors.Raw(variables_of_interest=np.array([0]), period=1.0),)
    )
    sim.simulation_length = 15000
    sim.configure()

    try:
        output = sim.run()[0]
        raw_data = output[1]
        print("Raw data shape before squeeze:", raw_data.shape)

        raw_data = np.squeeze(raw_data)
        print("Raw data shape after squeeze:", raw_data.shape)

        if raw_data.ndim == 2 and raw_data.shape[1] == conn_lesion.number_of_regions:
            ts = raw_data[:, 18]  
            alpha_power = np.mean(np.abs(np.fft.rfft(ts))**2)

            fc_matrix = np.corrcoef(raw_data.T)
            np.save(os.path.join(output_dir, f"fc_matrix_{i}.npy"), fc_matrix)
            mean_fc = np.mean(fc_matrix[np.triu_indices(fc_matrix.shape[0], k=1)])
        else:
            print(f"Unexpected shape for subject {i+1}. Skipping.")
            alpha_power = np.nan
            mean_fc = np.nan

    except Exception as e:
        print(f"Error for subject {i+1}: {e}")
        alpha_power = np.nan
        mean_fc = np.nan

    results.append({
        "subject_id": f"tbi_{i+1}",
        "alpha_power_dlpfc": alpha_power,
        "mean_fc_strength": mean_fc
    })

Running TBI subject 1...
2025-05-15 10:38:36,631 - ERROR - tvb.basic.readers - Could not import tvb_data Python module for default data-set!
Traceback (most recent call last):
  File "/opt/anaconda3/envs/tvb_env/lib/python3.10/site-packages/tvb/basic/readers.py", line 222, in try_get_absolute_path
    module_import = importlib.import_module(relative_module)
  File "/opt/anaconda3/envs/tvb_env/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 992, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 100

In [24]:
df = pd.DataFrame(results)
df.to_csv(os.path.join(output_dir, "tbi_strong_group_metrics.csv"), index=False)
print("All TBI simulations completed. Results saved to:", output_dir)

All TBI simulations completed. Results saved to: tbi_stronggroup_results
